In [9]:
# imports 
import pandas as pd
import seaborn as sns

In [10]:
# load the dataset
df_energy = pd.read_csv("..\data\energy_dataset.csv")
df_weather = pd.read_csv("..\data\weather_features.csv")
df_weather = df_weather[df_weather["city_name"] == "Valencia"]
df_weather.drop(columns=["city_name"], inplace=True)